# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

We're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (a ways to speed up your regular Python function)
* Using GPUs with TensorFlow (or TPUs)
* Exercise to try yourself!

## Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimension of our vector
vector.ndim

1

In [6]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10,7],[7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([[10.,7.],
                              [3.,2.],
                              [8.,9.]], dtype=tf.float16) # specify the data type with paramater
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# What's the dimension of another_matrix?
another_matrix.ndim

2

In [10]:
# Lets a create a tensor
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                      [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

What we've created so far:
* Scalar : a single number
* Vector : a number with direction (eg. wind,speed,direction)
* Matrix : a 2-dimensional array of numbers
* Tensor : a n-dimensional array of numbers (when n can be any number, a 0-dim tensor is scalar,1-dim tensor is a vector)

### Create tensor with `tf.Variable`

In [12]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [13]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: ignored

In [14]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [15]:
# Let's try change on unchangeable_tensor
unchangeable_tensor[0] = 7

TypeError: ignored

In [16]:
unchangeable_tensor[0].assign(7)

AttributeError: ignored

🔑 **Note:** Rarely in practic will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors of some abitary size which contain random numbers.

In [17]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1,random_2,random_1==random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [18]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
not_shuffled.ndim

2

In [19]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [20]:
# Shuffle out non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [21]:
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled,seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [22]:
tf.random.shuffle(not_shuffled,seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

🛠 **Exercise** : Read through TensorFlow documentation on random seed gemeration: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them

In [23]:
# 5 random tensors
first = tf.random.Generator.from_seed(40)
first = first.normal(shape=(3,2))
second = tf.random.Generator.from_seed(41)
second = second.normal(shape=(3,2))
third = tf.random.Generator.from_seed(42)
third = third.normal(shape=(3,3))
forth = tf.random.Generator.from_seed(42)
forth = forth.normal(shape=(2,3))
fifth = tf.random.Generator.from_seed(45)
fifth = fifth.normal(shape=(4,4))

In [24]:
first

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.78953624,  0.53897345],
       [-0.48535708,  0.74055266],
       [ 0.31662667, -1.4391748 ]], dtype=float32)>

In [25]:
second

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.31662667, -1.4391748 ],
       [ 0.58923835, -1.4268045 ],
       [-0.7565803 , -0.06854702]], dtype=float32)>

In [26]:
third

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702,  0.07595026],
       [-1.2573844 , -0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646, -0.7535805 ]], dtype=float32)>

In [27]:
forth

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702,  0.07595026],
       [-1.2573844 , -0.23193763, -1.8107855 ]], dtype=float32)>

In [28]:
fifth

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-0.3522796 ,  0.40621263, -1.0523509 ,  1.2054597 ],
       [ 1.6874489 , -0.4462975 , -2.3410842 ,  0.99009085],
       [-0.0876323 , -0.635568  , -0.6161736 , -1.9441465 ],
       [-0.48293006, -0.52447474, -1.0345329 ,  1.3066901 ]],
      dtype=float32)>

In [29]:
# shuffle them
tf.random.shuffle(first)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.48535708,  0.74055266],
       [ 0.31662667, -1.4391748 ],
       [ 0.78953624,  0.53897345]], dtype=float32)>

In [30]:
tf.random.set_seed(40) # global level random seed
tf.random.shuffle(second,seed=40) # operation level global seed

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.31662667, -1.4391748 ],
       [ 0.58923835, -1.4268045 ]], dtype=float32)>

In [31]:
tf.random.shuffle(third)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-1.2573844 , -0.23193763, -1.8107855 ],
       [-0.7565803 , -0.06854702,  0.07595026],
       [ 0.09988727, -0.50998646, -0.7535805 ]], dtype=float32)>

In [32]:
tf.random.shuffle(forth)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-1.2573844 , -0.23193763, -1.8107855 ],
       [-0.7565803 , -0.06854702,  0.07595026]], dtype=float32)>

In [33]:
tf.random.shuffle(fifth)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 1.6874489 , -0.4462975 , -2.3410842 ,  0.99009085],
       [-0.48293006, -0.52447474, -1.0345329 ,  1.3066901 ],
       [-0.3522796 ,  0.40621263, -1.0523509 ,  1.2054597 ],
       [-0.0876323 , -0.635568  , -0.6161736 , -1.9441465 ]],
      dtype=float32)>

### Other ways to make tensors

In [35]:
# Create a tensor of all ones
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [37]:
# Create a tensor of all zeros
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference betweeon NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [38]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1,25,dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [42]:
A = tf.constant(numpy_A,shape=(2,3,4))
B = tf.constant(numpy_A)
A,B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)